In [1]:
%load_ext autoreload
%autoreload 1

import sys
sys.path.append("../../utils/")

import matplotlib.pyplot as plt
%matplotlib inline
import pdb
import requests
import re

import pandas as pd

import networkx as nx

import signal

import warnings
warnings.filterwarnings("ignore")

from wiki_intro_scrapper import WikiIntroScrapper
from WikiMultiQuery import wiki_multi_query
from graph_helpers import create_dispersion_df, sort_dict_values

%aimport wiki_intro_scrapper
%aimport WikiMultiQuery

In [208]:
class GraphCreator:

    def __init__(self, entry):
        self.graph = nx.DiGraph()

        self.entry = entry

        wis = WikiIntroScrapper(f"https://en.wikipedia.org/wiki/{entry}")
        wis.parse_intro_links()

        self.intro_nodes = wis.intro_link_titles

        self.visited = {entry}
        self.next_links = []

        self.query_articles([entry])

        # setup timeout function

        def handle_alarm(signum, frame):
            raise RuntimeError

        signal.signal(signal.SIGALRM, handle_alarm)

    def add_edges(self, articles):
        for article in articles:
            self.graph.add_edges_from(
                [(article['title'], link) for link in article['links']])
            self.graph.add_edges_from(
                [(linkhere, article['title']) for linkhere in article['linkshere']])

    def plot_graph(self):
        nx.draw(self.graph)
        plt.show()

    def get_degrees(self):
        return sort_dict_values(dict(self.graph.degree()), ["node", "degree"], "degree",)

    def get_centrality(self):
        return sort_dict_values(nx.eigenvector_centrality(self.graph), ["node", "centrality"], "centrality")

    def get_dispersion(self, nodes=None, comparison_node=None):
        if not nodes:
            nodes = self.graph.nodes
        if not comparison_node:
            comparison_node = self.entry

        return create_dispersion_df(gc.graph, comparison_node, nodes).sort_values("dispersion", ascending=False).reset_index().drop("index", axis=1)

    def get_pageranks(self):
        page_ranks = sorted([(key, value) for key, value in nx.algorithms.link_analysis.pagerank(
            self.graph).items()], key=lambda x: x[1], reverse=True)
        return pd.DataFrame(page_ranks, columns=["node", "page_rank"])

    def get_reciprocity(self):
        return sort_dict_values(nx.algorithms.reciprocity(self.graph, self.graph.nodes), ['node', 'reciprocity'], 'reciprocity')

    def get_adjusted_reciprocity(self):
        r = self.get_reciprocity()
        d = self.get_degrees()

        r_d = r.merge(d, on="node", how="inner")
        r_d['adjusted_reciprocity'] = rec_deg.reciprocity * rec_deg.degrees

        adjusted_reci = r_d.sort_values("adjusted_reciprocity", ascending=False)
        return adjusted_reci.reset_index().drop(["degree", "reciprocity", "index"], axis=1)
    
    def get_shortes_path(self, source=None, ascending=False):
        if not source:
            source = self.entry
            
        paths = nx.algorithms.single_source_shortest_path_length(self.graph, source)
        return sort_dict_values(paths, ["node", "shortest_path_length_from_source"], "shortest_path_length_from_source", ascending=ascending)
    
    def create_ego(self, node=None):
        if not node:
            node = self.entry

        ego = nx.ego_graph(self.graph, node)
        ego.name = node
        return ego

    def expand_network(self, group_size=10, timeout=10):

        num_links = len(self.next_links)

        link_group = []

        for i in range(num_links):
            link = self.next_links.pop(0)
            if not link in self.visited:

                link_group.append(link)

                if len(link_group) == group_size or (i == num_links - 1 and len(link_group) > 0):
                    print("{:.2%}".format(i/num_links))
                    try:
                        signal.alarm(timeout)
                        self.visited.update(link_group)
                        self.query_articles(link_group)
                        signal.alarm(0)
                        link_group = []
                    except:
                        link_group = []
                        continue
        signal.alarm(0)

    def update_next_links(self, articles):
        for article in articles:
            for link in article['links']:
                self.next_links.append(link)

    def query_articles(self, titles, generate_graph=True):
        articles = wiki_multi_query(titles)

        self.update_next_links(articles)
        self.add_edges(articles)

## TESTS

In [233]:
dom_dict = nx.algorithms.dominance.immediate_dominators(gc.graph, start=gc.entry)

dom_counts = {}

for key, value in dom_dict.items():
    if value in dom_counts:
        dom_counts[value] += 1
    else:
        dom_counts[value] = 1
        
dom_counts

{'Decision tree': 2680,
 'Algorithm': 255,
 'Causality': 498,
 'Causal model': 41,
 'Probability': 173,
 'Conditional probability': 30,
 'Expected value': 102,
 'Time': 447,
 'Timeline': 59,
 'Morphological analysis (problem-solving)': 6,
 'Argument map': 51,
 'Concept map': 19,
 'Business decision mapping': 3,
 'Operations management': 482,
 'Markov chain': 337,
 'Graph drawing': 65,
 'DRAKON': 35,
 'Flowchart': 54,
 'Topic map': 93,
 'Cognitive map': 42,
 'Dendrogram': 16,
 'Data visualization': 49,
 'Geovisualization': 12,
 'Infographic': 106,
 'Visualization (graphics)': 124,
 'Decision support system': 102,
 'Decision analysis': 16,
 'Diminishing returns': 30,
 'Utility': 152,
 'Expected utility hypothesis': 46,
 'Operations research': 236,
 'Topological combinatorics': 25,
 'Influence diagram': 13,
 'Decision tree learning': 29,
 'Association rule learning': 31,
 'Boosting (machine learning)': 45,
 'Random forest': 30,
 'Information gain in decision trees': 14,
 'ID3 algorithm': 

In [210]:
gc = GraphCreator("Decision tree")

In [211]:
gc.expand_network(group_size=2, timeout=5)

1.14%
3.41%
5.68%
7.95%
10.23%
12.50%
14.77%
17.05%
19.32%
21.59%
23.86%
26.14%
28.41%
30.68%
32.95%
35.23%
37.50%
39.77%
42.05%
44.32%
46.59%
48.86%
51.14%
53.41%
55.68%
57.95%
60.23%
62.50%
64.77%
67.05%
69.32%
71.59%
73.86%
76.14%
78.41%
80.68%
82.95%
85.23%
87.50%
89.77%
92.05%
94.32%
96.59%
98.86%


In [212]:
gc.get_degrees().head(25)

,node,degree
0,Algorithm,3377
1,Time,2898
2,Causality,2281
3,Probability,1866
4,Operations research,1853
5,Semantic Web,1387
6,Flowchart,1289
7,Topic map,1232
8,Utility,1221
9,Expected value,969


In [213]:
gc.get_centrality().head(25)

,node,centrality
0,Topic map,0.324205
1,Flowchart,0.270652
2,Time,0.216060
3,Causality,0.196145
4,Timeline,0.162259
5,Semantic Web,0.160591
6,Graph drawing,0.128667
7,Semantic network,0.118826
8,Hypertext,0.117326
9,Mind map,0.111488


In [216]:
gc.get_shortes_path().head(25)

,node,shortest_path_length_from_source
0,Trickle,4
1,Garrett Birkhoff,4
2,Correspondence analysis,4
3,Digital Bibliography & Library Project,4
4,Dual (math),4
5,Extension (semantics),4
6,Formal logic,4
7,Galois connection,4
8,Heterogeneous relation,4
9,Lake,4


In [ ]:
gc.get_dispersion().head(25)

In [ ]:
gc.get_pageranks().head(25)

In [ ]:
gc.get_adjusted_reciprocity().head(25)

In [ ]:
ego = nx.ego_graph(gc.graph, "Regression analysis", undirected=True)
ego.name = "Regression analysis"
print(nx.info(ego))

In [ ]:
len(ego.nodes)

In [ ]:
nx.draw(ego)
plt.show()

In [ ]:
sorted([(key, value) for key, value in nx.eigenvector_centrality(ego).items()], key=lambda x: x[1], reverse=True)

In [ ]:
def sort_dict_values(dict, columns, sort_column, ascending=False):
    to_list = [(key, value) for key, value in dict.items()]
    return pd.DataFrame(to_list, columns=columns).sort_values(sort_column, ascending=ascending).reset_index().drop("index", axis=1)

In [ ]:
regression_ego = sort_dict_values(nx.betweenness_centrality(ego), ["node", "betweenness"], "betweenness")
regression_ego
# regression_ego[regression_ego.node == "Random forest"]